In [101]:
import sys
import tensorflow as tf
import glob
import PIL
import cv2
from sklearn.utils import shuffle
import numpy as np
from PIL import Image


In [102]:
from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[u'/gpu:0']

In [103]:
data_dir = "/home/p2/skripsie/data"
nb_images = len(glob.glob(data_dir+"/*/*.*"))
hold = 0
image_size = 256
test_percentage = .2

print nb_images
# 52803

X = []
# X.resize(nb_images)
Y = []
class_names = []
# print X.shape

#data folder structure
#Dataset->[ABR->[ABR1]; AS->[AS1; AS2]]
directory = data_dir + "/*"
classes = glob.glob(directory)
label = 0
imgcount = 0
for one_class in classes:
    class_names.append(one_class.split('/')[-1])
    imgs = glob.glob(one_class+'/*')
    print label ," ", one_class
    for img in imgs:
        imgcount += 1
#         im = Image.open(img)
#         im_arr = np.array(im) # img = file path
#         print img
        X.append(img)
        Y.append(label)
    label += 1
X = np.array(X)
Y = np.array(Y)
# print Y
print class_names
print imgcount
print np.shape(X)

54305
0   /home/p2/skripsie/data/Apple___Apple_scab
1   /home/p2/skripsie/data/Apple___Black_rot
2   /home/p2/skripsie/data/Apple___Cedar_apple_rust
3   /home/p2/skripsie/data/Apple___healthy
4   /home/p2/skripsie/data/Blueberry___healthy
5   /home/p2/skripsie/data/Cherry_(including_sour)___healthy
6   /home/p2/skripsie/data/Cherry_(including_sour)___Powdery_mildew
7   /home/p2/skripsie/data/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
8   /home/p2/skripsie/data/Corn_(maize)___Common_rust_
9   /home/p2/skripsie/data/Corn_(maize)___healthy
10   /home/p2/skripsie/data/Corn_(maize)___Northern_Leaf_Blight
11   /home/p2/skripsie/data/Grape___Black_rot
12   /home/p2/skripsie/data/Grape___Esca_(Black_Measles)
13   /home/p2/skripsie/data/Grape___healthy
14   /home/p2/skripsie/data/Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
15   /home/p2/skripsie/data/Orange___Haunglongbing_(Citrus_greening)
16   /home/p2/skripsie/data/Peach___Bacterial_spot
17   /home/p2/skripsie/data/Peach___healthy
18 

In [104]:
print X[2000]


/home/p2/skripsie/data/Apple___healthy/430fd7d8-9390-411c-b849-e421cf11f7a7___RS_HL 7314.JPG


In [105]:
#Next up: shuffle data and split up into train and test sets.
#shuffle
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def shuffle_in_unison_scary(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

# X, Y = unison_shuffled_copies(X, Y)
shuffle_in_unison_scary(X, Y)


print X[0]
print Y[0]


#split in train and test --> 

test_nr = (int)(test_percentage*nb_images)

print test_nr
X_train = X[:test_nr]
X_test = X[test_nr:]
Y_train = Y[:test_nr]
Y_test = Y[test_nr:]

print X_train.shape
print X_test.shape
print Y_train.shape
print Y_test.shape

/home/p2/skripsie/data/Tomato___Tomato_Yellow_Leaf_Curl_Virus/fafa446d-bbd4-4fe2-a0a8-72472392548e___UF.GRC_YLCV_Lab 03233.JPG
37
10861
(10861,)
(43444,)
(10861,)
(43444,)


need:
    batch_X and batch_Y train data
    batch_X, batch_Y = next_batch(38*2) <-- function to get them
    
    replacement for this:
    test_data={X:mnist.test.images, Y_:mnist.test.labels}

In [106]:
def toImages(batchFiles):    
    imgBatch = []
    
    for imgfile in batchFiles:
        image = cv2.imread(imgfile)
        image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
        image = image.astype(np.float32)
        image = np.multiply(image, 1.0 / 255.0)
        
        imgBatch.append(image)
    
    imgBatch = np.array(imgBatch)
#     print imgBatch.shape
    return imgBatch

In [107]:
batch_Files = X_train[0 * 5:(0+1) * 5]
batch_X = toImages(batch_Files)
# print a
# print b

In [111]:
EPOCH = 100
BATCH_SIZE = 128
NR_CLASSES = 38
TRAIN_DATASIZE = nb_images
PERIOD = TRAIN_DATASIZE/BATCH_SIZE #Number of iterations for each epoch


tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 196606])   # A placeholder for the input image
X_reshaped = tf.reshape(X, [-1, 256, 256, 3])   # TensorFlow's convolutional operation wants a "volume" 

layer_1_depth = 2   # How deep is layer 1?
layer_2_depth = 4   # How deep is layer 2? 
filter_size = 5     # What size of filters do we want?

# Create the parameters for layer 1
W1 = tf.Variable(tf.truncated_normal([filter_size, filter_size, 3, layer_1_depth] ,stddev=0.1))
B1 = tf.Variable(tf.ones([layer_1_depth])/NR_CLASSES)

# Create the parameters for layer 2
W2 = tf.Variable(tf.truncated_normal([filter_size, filter_size, layer_1_depth, layer_2_depth] ,stddev=0.1))
B2 = tf.Variable(tf.ones([layer_2_depth])/NR_CLASSES)

# CONVOLUTIONAL LAYER 1: 
Y1 = tf.nn.relu(tf.nn.conv2d(X_reshaped, W1, strides=[1, 1, 1, 1], padding='SAME') + B1)

# POOLING 
pool = tf.nn.max_pool(Y1, ksize=[1, 2, 2, 1],
                      strides=[1, 2, 2, 1], 
                      padding='SAME')

# CONVOLUTIONAL LAYER 2
Y2 = tf.nn.relu(tf.nn.conv2d(pool, W2, strides=[1, 2, 2, 1], padding='SAME') + B2)

print "Shape of Y2: ", Y2.get_shape()

fully_connected_size = 1444

# Create the parameters for the hidden fully connected layer
W3 = tf.Variable(tf.truncated_normal([7*7*layer_2_depth, fully_connected_size] ,stddev=0.1))
B3 = tf.Variable(tf.ones([fully_connected_size])/NR_CLASSES)

# HIDDEN FULLY CONNECTED LAYER:
YY = tf.contrib.layers.flatten(Y2)  # First we need to flatten the volume
Y3 = tf.nn.relu(tf.matmul(YY, W3) + B3)  # Now compute the hidden output

print "Shape of Y3: ", Y3.get_shape()

# Create the parameters of the final logits layer
W4 = tf.Variable(tf.zeros([fully_connected_size, NR_CLASSES]))  
B4 = tf.Variable(tf.zeros([NR_CLASSES]))

# Compute the model predictions! 
logits = tf.matmul(Y3, W4) + B4   # Compute the logits
Y = tf.nn.softmax(logits)     # Compute the model predictions

# placeholder for correct answers
Y_ = tf.placeholder(tf.float32, [None, NR_CLASSES])   

# loss function
cross_entropy = -tf.reduce_mean(Y_ * tf.log(Y))

# % of correct answers found in batch
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

optimizer = tf.train.AdamOptimizer(0.003)   # Now we try an Adam optimizer
train_step = optimizer.minimize(cross_entropy)   # An op to minimise the cross entropy loss

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

avg_train_cost = 0.
avg_train_accuracy = 0.
avg_test_cost = 0.
avg_test_accuracy = 0.

###############################################################

for e in range(EPOCH):
    idxs = numpy.random.permutation(TRAIN_DATASIZE) #shuffled ordering
    X_random = X_train[idxs]
    Y_random = Y_train[idxs]
    for i in range(PERIOD):   
        batch_Y = Y_random[i * BATCH_SIZE:(i+1) * BATCH_SIZE]
        batch_Files = X_random[i * BATCH_SIZE:(i+1) * BATCH_SIZE]
        batch_X = toImages(batch_Files)
        train_data = {X: batch_X, Y:batch_Y}
        sess.run(train_step,feed_dict = train_data)

###############################################################

# num_steps = 5000

# for i in range(num_steps):
#     # load batch of images and correct answers
#     batch_X, batch_Y = next_batch(38*2)
#     train_data={X: batch_X, Y_: batch_Y}

#     # train
#     sess.run(train_step, feed_dict=train_data)

   
    
###############################################################    
     
    # success ? add code to print it
    a,c = sess.run([accuracy, cross_entropy], feed_dict=train_data)
    
    if i % 100 == 0:
        print "Step", i, "Current train cost and accuracy: ", c, a
    
    avg_train_cost += c
    avg_train_accuracy += a
    
print("Completed Training")

avg_train_cost /= num_steps
avg_train_accuracy /= num_steps

print("Average train cost: ", avg_train_cost)
print("Average train accuracy: ", avg_train_accuracy)


# success on test data ?
test_data={X:toImages(X_test), Y_:Y_test}
a,c = sess.run([accuracy, cross_entropy], feed_dict=test_data)

print("Test cost: ", c)
print("Test accuracy: ", a)



# need:
#     batch_X and batch_Y train data
#     batch_X, batch_Y = next_batch(38*2) <-- function to get them
    
#     replacement for this
#     test_data={X:X_test, Y_:Y_test}

Shape of Y2:  (?, 64, 64, 4)


ValueError: Dimensions must be equal, but are 16384 and 196 for 'MatMul' (op: 'MatMul') with input shapes: [?,16384], [196,1444].